# 05 – Tests (golden + contract)


Lightweight assertions you can run in-order.


In [ ]:
using semantic_kernel_app;
using Microsoft.Extensions.Logging;

var logger = LoggerFactory.Create(b=>b.AddConsole());
var clf = new RuleIntentClassifier(logger.CreateLogger<RuleIntentClassifier>());
var sf = new SlotFiller(logger.CreateLogger<SlotFiller>());
var disp = new QueryDispatcher(new InMemoryEmployeeApi(
    new Microsoft.Extensions.Caching.Memory.MemoryCache(new Microsoft.Extensions.Caching.Memory.MemoryCacheOptions()), new AppConfig()),
    logger.CreateLogger<QueryDispatcher>());

void Assert(bool cond, string msg){ if(!cond) throw new Exception("ASSERT: " + msg); }

// Classifier golden tests
var (i1, c1) = clf.Classify("What are Rick and Summer's emails?");
Assert(i1==Intent.GetContactInfo, "classifier email intent");

// Slot filler
var (slots, conf, clar) = sf.Fill("Managers in engineering hired after 2020");
Assert(slots.Operator=="after", "slots operator after");
Assert(slots.Department=="engineering", "slots dept engineering");

// Dispatcher contract
var spec = new QuerySpec(Intent.FilterByHireDate, new Slots(Department:"engineering", Operator:"after", Date:new DateTime(2020,1,1)));
var result = (List<Employee>) await disp.ExecuteAsync(spec);
Assert(result.All(e=> e.Department=="Engineering" && e.OriginalHireDate>new DateTime(2020,1,1)), "dispatch filter combo");
Console.WriteLine("All tests passed.");
